In [34]:
!python --version

Python 3.10.14


## Importing Libraries

In [35]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split

# Reading The Dataset

In [36]:
PROJECT_DIR = r"C:\Users\abhid\Desktop\campusx\flightprice_sagemaker"
DATA_DIR = "data"

In [37]:
os.path.join(PROJECT_DIR,DATA_DIR)

'C:\\Users\\abhid\\Desktop\\campusx\\flightprice_sagemaker\\data'

In [38]:
def get_data(name):
    """
    user will give name of the csv file
    """
    filename = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR,DATA_DIR,filename)

    return pd.read_csv(file_path)


In [39]:
flights = get_data("flight_price")

In [40]:
flights.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882


# Preliminary Checks

In [41]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [42]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

# Check for Duplicates

In [43]:
flights.duplicated().sum()

220

In [44]:
# Viweing the duplicate rows
flights[flights.duplicated(keep = False)].sort_values(["Airline","Date_of_Journey","Route","Dep_Time"])

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
1535,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


## Observations

## Detailed Analysis

In [45]:
# airline
flights['Airline'].value_counts()

Airline
Jet Airways                          3849
IndiGo                               2053
Air India                            1752
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: count, dtype: int64

In [46]:
flights['Airline'].str.replace("Premium economy","").str.replace("Business","").str.title().unique()

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Vistara ',
       'Jet Airways ', 'Multiple Carriers ', 'Trujet'], dtype=object)

In [47]:
# source
print(flights['Source'].unique())
print(flights['Destination'].unique())

['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']
['New Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Delhi' 'Hyderabad']


In [48]:
# check if Dep time entries are valid, if they contain character other than digits and column
flights['Dep_Time'].str.contains("[^0-9:]").sum()

0

## So, all values are in hh:mm format as desrired

In [49]:
# convert Dep Time to tie format
pd.to_datetime(flights['Dep_Time']).dt.time

C:\Users\abhid\AppData\Local\Temp\ipykernel_8144\2877393476.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights['Dep_Time']).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

In [50]:
# check if Dep time entries are valid, if they contain character other than digits and column
flights['Arrival_Time'].str.contains("[^0-9:]")

0         True
1        False
2         True
3        False
4        False
         ...  
10678    False
10679    False
10680    False
10681    False
10682    False
Name: Arrival_Time, Length: 10683, dtype: bool

In [51]:
pd.to_datetime(flights['Arrival_Time'].str.split(" ",n=1).str.get(0)).dt.time

C:\Users\abhid\AppData\Local\Temp\ipykernel_8144\1518000786.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights['Arrival_Time'].str.split(" ",n=1).str.get(0)).dt.time


0        01:10:00
1        13:15:00
2        04:25:00
3        23:30:00
4        21:35:00
           ...   
10678    22:25:00
10679    23:20:00
10680    11:20:00
10681    14:10:00
10682    19:15:00
Name: Arrival_Time, Length: 10683, dtype: object

In [52]:
# Anomaly in duration column
flights[~flights['Duration'].str.contains("h")]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


### We need to drop this error row of duration 5 minutes

# converting duration to minutes


In [53]:
flights['Duration'].str.split(" ",expand = True)[0].str.replace("h","").fillna("0")

0         2
1         7
2        19
3         5
4         4
         ..
10678     2
10679     2
10680     3
10681     2
10682     8
Name: 0, Length: 10683, dtype: object

In [54]:
flights['Duration'].str.split(" ",expand = True)[1].str.replace("m","").fillna("0").astype(int)

0        50
1        25
2         0
3        25
4        45
         ..
10678    30
10679    35
10680     0
10681    40
10682    20
Name: 1, Length: 10683, dtype: int32

In [55]:
flights['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [56]:
flights['Total_Stops'].str.replace("non-stop","0").str.replace(" stops?", "", regex=True)

0        0
1        2
2        2
3        1
4        1
        ..
10678    0
10679    0
10680    0
10681    0
10682    2
Name: Total_Stops, Length: 10683, dtype: object

In [57]:
flights[flights['Total_Stops'].isna()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [58]:
flights.select_dtypes(include='O').columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info'],
      dtype='object')

# Cleaning Operations

In [59]:
def cleaned_df(df):
    """
    Returns cleaned dataframe
    """
    clean_df = df.copy()

    # dropping route column
    clean_df.drop(columns = ['Route'],inplace = True)

    # dropping erroneous duration row
    clean_df.drop(index = [6474,9039],inplace=True)

    # dropping duplicates
    clean_df.drop_duplicates(inplace = True)

    # Replace Delhi with New Delhi in Destination
    clean_df['Destination'] = clean_df['Destination'].replace("Delhi", "New Delhi")

    # Removing Business and Premium Economy labels from airlines names, Titlecase the labels
    clean_df['Airline'] = clean_df['Airline'].str.replace("Premium economy","").str.replace("Business","").str.title()

    # removing extra spaces using strip() using assigns and dict comprehension
    str_cols = clean_df.select_dtypes(include='object').columns
    clean_df = clean_df.assign(**{col: clean_df[col].str.strip() for col in str_cols})

    # Fixing datetime of Date_of_Journey column
    clean_df['Date_of_Journey'] = pd.to_datetime(clean_df['Date_of_Journey'],dayfirst = True)

    # Converting Dep Time to time format
    clean_df['Dep_Time'] = pd.to_datetime(clean_df['Dep_Time']).dt.time

    # Fixing Arrival time
    clean_df['Arrival_Time'] = pd.to_datetime(clean_df['Arrival_Time'].str.split(" ",n=1).str.get(0)).dt.time

    # Converting duration to minutes
    clean_df['dur_hr'] = clean_df['Duration'].str.split(" ",expand = True)[0].str.replace("h","").fillna("0").astype(int)
    clean_df['dur_min'] = clean_df['Duration'].str.split(" ",expand = True)[1].str.replace("m","").fillna("0").astype(int)

    clean_df['duration_min'] = clean_df['dur_hr']*60 + clean_df['dur_min']

    # Converting Total Stops to integer
    clean_df['Total_Stops'] = pd.to_numeric(clean_df['Total_Stops'].str.replace("non-stop","0").str.replace(" stops?", "", regex=True))

    # Additional info
    clean_df['Additional_Info'] = clean_df['Additional_Info'].replace("No info", "No Info")
    
    # dropping intermediate columns created
    clean_df.drop(columns = ['dur_min','dur_hr','Duration'],inplace = True)
     
    
    # lowercase of column names
    clean_df = clean_df.rename(columns = str.lower)
    
    return clean_df

In [60]:
flights_cleaned = cleaned_df(flights)

C:\Users\abhid\AppData\Local\Temp\ipykernel_8144\3190716208.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  clean_df['Dep_Time'] = pd.to_datetime(clean_df['Dep_Time']).dt.time
C:\Users\abhid\AppData\Local\Temp\ipykernel_8144\3190716208.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  clean_df['Arrival_Time'] = pd.to_datetime(clean_df['Arrival_Time'].str.split(" ",n=1).str.get(0)).dt.time


In [61]:
flights_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10461 entries, 0 to 10682
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   airline          10461 non-null  object        
 1   date_of_journey  10461 non-null  datetime64[ns]
 2   source           10461 non-null  object        
 3   destination      10461 non-null  object        
 4   dep_time         10461 non-null  object        
 5   arrival_time     10461 non-null  object        
 6   total_stops      10461 non-null  int64         
 7   additional_info  10461 non-null  object        
 8   price            10461 non-null  int64         
 9   duration_min     10461 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(2), object(6)
memory usage: 858.1+ KB


In [62]:
flights_cleaned.sample(5)

,airline,date_of_journey,source,destination,dep_time,arrival_time,total_stops,additional_info,price,duration_min
5768,Multiple Carriers,2019-05-09,Delhi,Cochin,08:45:00,19:00:00,1,No Info,9794,615
8395,Spicejet,2019-06-06,Kolkata,Banglore,17:10:00,19:40:00,0,No check-in baggage included,3841,150
1986,Indigo,2019-06-27,Chennai,Kolkata,05:15:00,07:40:00,0,No Info,3540,145
4959,Jet Airways,2019-06-09,Kolkata,Banglore,17:00:00,22:05:00,1,No Info,12692,1745
826,Jet Airways,2019-03-01,Banglore,New Delhi,05:45:00,16:10:00,1,No Info,26890,625


In [63]:
flights_cleaned['additional_info'].value_counts()

additional_info
No Info                         8184
In-flight meal not included     1926
No check-in baggage included     318
1 Long layover                    19
Change airports                    7
Business class                     4
1 Short layover                    1
Red-eye flight                     1
2 Long layover                     1
Name: count, dtype: int64

### Saving processed file

In [64]:
def export_to_csv(df,name):
    filename = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR,DATA_DIR,filename)
    
    df.to_csv(file_path,index = False)

In [65]:
export_to_csv(flights_cleaned,"flights_cleaned")

## Split the Data

In [66]:
X = flights_cleaned.copy().drop('price',axis = 1)
y = flights_cleaned['price'].copy()

In [67]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(6694, 9) (6694,)
(1674, 9) (1674,)
(2093, 9) (2093,)


## Export the Data Subsets

In [68]:
X_train.join(y_train).head(3)  # by default joins on index

,airline,date_of_journey,source,destination,dep_time,arrival_time,total_stops,additional_info,duration_min,price
3832,Jet Airways,2019-05-27,Delhi,Cochin,20:55:00,12:35:00,1,In-flight meal not included,940,12898
3095,Jet Airways,2019-06-12,Kolkata,Banglore,18:55:00,16:20:00,1,No Info,1285,13044
9717,Air India,2019-05-18,Delhi,Cochin,09:45:00,09:25:00,2,No Info,1420,10975


In [69]:
export_to_csv(X_train.join(y_train),'train')

In [70]:
export_to_csv(X_val.join(y_val),'val')

In [71]:
export_to_csv(X_test.join(y_test),'test')